In [5]:
import cv2
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

# Define the global dataframe
output_df = pd.DataFrame(columns=['Frame', 'ghost_X', 'ghost_Y', 'Ghost_ID'])

def localize_ghost(image_path, ghost_colors=[(200, 72, 72), (146, 70, 192), (84, 184, 153), (111, 111, 111), (66, 114, 194), (65, 158, 129), (57, 102, 186), (197, 89, 179), (211, 211, 211)], background_color=(228, 111, 111), color_tolerance=10, min_contour_area=10, max_contour_area=100, exclude_bottom_pixels=40, max_aspect_ratio=2.0):
    global output_df  # Use the global dataframe

    # Load the image
    image = cv2.imread(image_path)

    # Exclude the bottom pixels from the image
    height, width, _ = image.shape
    image = image[:-exclude_bottom_pixels, :]

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Iterate over each ghost color
    for ghost_id, ghost_color in enumerate(ghost_colors):
        # Create a mask for the current ghost color
        lower_ghost = np.array([c - color_tolerance for c in ghost_color], dtype=np.uint8)
        upper_ghost = np.array([c + color_tolerance for c in ghost_color], dtype=np.uint8)
        ghost_mask = cv2.inRange(image_rgb, lower_ghost, upper_ghost)

        # Find contours in the ghost mask
        contours, _ = cv2.findContours(ghost_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours based on area, aspect ratio, and max contour area
        for contour in contours:
            area = cv2.contourArea(contour)
            if min_contour_area < area < max_contour_area:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h if h != 0 else 0

                if aspect_ratio < max_aspect_ratio:
                    # Calculate the centroid (center) of the ghost object
                    centroid_x = x + w // 2
                    centroid_y = y + h // 2

                    # Add the frame, localization, and ghost ID to the dataframe
                    output_df = pd.concat([output_df, pd.DataFrame([{'Frame': os.path.basename(image_path), 'ghost_X': centroid_x, 'ghost_Y': centroid_y, 'Ghost_ID': ghost_id}])], ignore_index=True)

                    # Draw a marker at the estimated position
                    cv2.circle(image, (centroid_x, centroid_y), 5, (255, 0, 0), -1)

    # Display the result
    # cv2.imshow('Ghost Localization', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

def process_frames(folder_path, output_name):
    # Get the total number of files for the progress bar
    total_files = len([filename for filename in os.listdir(folder_path) if filename.endswith('.png')])

    # Create a tqdm progress bar
    for filename in tqdm(sorted(os.listdir(folder_path)), total=total_files):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            localize_ghost(image_path)

    # Save the dataframe to a CSV file
    filename = f"ghost_localization_{output_name[10:]}.csv"
    output_df.to_csv(filename, index=False)

if __name__ == "__main__":
    # Replace 'your_folder_path' with the actual path to your folder containing PNG frames
    folders = ["ms_pacman/52_RZ_2394668_Aug-10-14-52-42", "ms_pacman/61_RZ_2737165_Aug-14-14-09-12", "ms_pacman/64_RZ_2809807_Aug-15-10-11-14", "ms_pacman/71_RZ_2901714_Aug-16-11-54-21",
               "ms_pacman/76_RZ_3010789_Aug-17-18-01-02",  "ms_pacman/89_RZ_3438636_Aug-22-16-59-30", "ms_pacman/91_RZ_3502739_Aug-23-10-43-13", "ms_pacman/105_RZ_3614646_Aug-24-17-47-26", 
               "ms_pacman/125_JAW_2679477_Dec-07-14-48-06", "ms_pacman/126_JAW_2682560_Dec-07-15-37-29", "ms_pacman/131_KM_3030302_Dec-11-16-12-47", "ms_pacman/137_KM_3115947_Dec-12-15-59-59",
               "ms_pacman/199_RZ_3968652_Dec-04-18-11-43", "ms_pacman/202_RZ_4051421_Dec-05-17-16-28", "ms_pacman/209_RZ_6964528_Jan-08-10-23-46", "ms_pacman/271_RZ_3101375_Mar-20-12-03-54", "ms_pacman/271_RZ_3101375_Mar-20-12-03-54",
               "ms_pacman/273_RZ_3279899_Mar-22-13-39-02", "ms_pacman/378_RZ_2184332_Jul-03-15-07-35", "ms_pacman/467_RZ_3314679_Jul-16-17-05-24", "ms_pacman/496_RZ_3560871_Jul-19-13-28-35"]
    for i in folders: 
        # Define the global dataframe
        output_df = pd.DataFrame(columns=['Frame', 'ghost_X', 'ghost_Y', 'Ghost_ID'])
        process_frames(i, i)


100%|██████████| 17793/17793 [00:25<00:00, 692.78it/s]
